# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2025-11-12 07:09:30] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-11-12 07:09:30] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-11-12 07:09:30] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-12 07:09:32] INFO trace.py:60: opentelemetry package is not installed, tracing disabled


[2025-11-12 07:09:33] WARNING server_args.py:1197: Attention backend not explicitly specified. Use fa3 backend by default.


...

Capturing batches (bs=120 avail_mem=76.30 GB):   5%|▌         | 1/20 [00:00<00:03,  5.84it/s]

Capturing batches (bs=72 avail_mem=76.27 GB):  35%|███▌      | 7/20 [00:00<00:00, 20.47it/s]

Capturing batches (bs=32 avail_mem=76.25 GB):  50%|█████     | 10/20 [00:00<00:00, 21.57it/s]

Capturing batches (bs=8 avail_mem=76.23 GB):  75%|███████▌  | 15/20 [00:01<00:00, 13.35it/s] 

Capturing batches (bs=1 avail_mem=76.22 GB): 100%|██████████| 20/20 [00:01<00:00, 16.73it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Brian and I am a big fan of all things musical! I have performed in the music of Baroque, Classical, Romantic, Jazz, Fusion, Hip Hop, Electronic, Pop, Metal, and World music, and I have taken on many different musical styles as a performer. I have also studied Music Theory, Jazz Studies, and various other musical genres. I have a degree in Music Education, and have been a professional music educator for over 10 years. I have been performing as a soloist, chamber group, and in a variety of musical ensembles for more than 10 years, including the Colorado Symphony Orchestra,
Prompt: The president of the United States is
Generated text:  a) a person b) an office c) a president's duty d) his job d) his job
The president of the United States is a person. The office that the president of the United States holds is an elected position. This position involves making decisions and taking action on behalf of the country, which requires a person to perfor

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [job title] at [company name]. I'm excited to meet you and learn more about your career. What can you tell me about yourself? I'm a [insert a short description of your personality or skills]. And what's your favorite hobby or activity? I enjoy [insert a hobby or activity]. And what's your favorite book or movie? I love [insert a favorite book or movie]. And what's your favorite place to go? I love [insert a favorite place]. And what's your favorite color? I love [insert a favorite color]. And what's your favorite food? I

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, also known as the City of Light. It is a historic city with a rich cultural heritage, renowned for its art, architecture, and cuisine. Paris is also a major financial center and a major tourist destination, attracting millions of visitors each year. The city is home to many famous landmarks, including the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. Paris is a vibrant and dynamic city that continues to evolve and grow, with a strong emphasis on innovation and creativity. The city is also known for its diverse population, with a mix of French, European, and international cultures. Overall, Paris is a city

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by rapid advancements in areas such as machine learning, natural language processing, and computer vision. Some potential trends include:

1. Increased integration with human intelligence: AI is likely to become more integrated with human intelligence, allowing machines to learn from and adapt to human behavior and preferences.

2. Enhanced privacy and security: As AI systems become more sophisticated, there will be increased concerns about privacy and security. There will be efforts to develop more secure and ethical AI systems.

3. Greater use of AI in healthcare: AI is already being used in healthcare to diagnose and treat diseases, and there is potential for even more advanced AI in



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name]. I'm a [Age] year old [Occupation]. I'm a [Your profession/character] and I'm passionate about [Your favorite hobby or skill]. Whether it's [Spend time with pets, travel, reading, etc.], I find it incredibly fulfilling to be [Your chosen hobby]. I'm a [Your friend's name] and I'm always here for you. [Name's a friend]. How can I help you today? [Name's a family member]. How can I help you today? [Name's a mentor]. How can I help you today? [Name's a teacher].

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, the city that hosts the Eiffel Tower and the Louvre Museum. Paris is a famous cultural and historical center with a rich history and diverse architecture. The city is also known for its world-renowned cuisine, fashion, and art scene. With its city center bustling with life and vibrant culture,

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

],

 and

...

 learn

 from

 data

 and

 adjust

In [6]:
llm.shutdown()